# Assignment 3

In [120]:
import pandas as pd
import requests
import urllib.request, urllib.parse, urllib.error
import re
from datetime import timedelta

api_key = 'AIzaSyCYNp4jAj_________6VSklFq_UnJI'

In [121]:
# read the text file into a dataframe
df = pd.read_csv("C:/Users/ariel/Desktop/מטלה 3/dests.txt", header = None)
df.columns = ['Target']

In [122]:
def Distance_Duration(destination):
    serviceurl="https://maps.googleapis.com/maps/api/distancematrix/json?"
    parms = dict() 
    parms['origins'] = 'Tel Aviv'
    parms['destinations'] = destination
    parms['key'] = api_key
    url = serviceurl + urllib.parse.urlencode(parms)
    
    try:
        response = requests.get(url)
        if not response.status_code == 200:
            print("HTTP error",response.status_code)
        else:
            try:
                response_data = response.json()
                seconds = response_data['rows'][0]['elements'][0]['duration']['value']
                minutes = (seconds//60)%60
                hours = (seconds//3600)
                text = str(hours) + ' hours ' + str(minutes) + ' minutes ' 
                return (response_data['rows'][0]['elements'][0]['distance']['value']/1000, text)

            
                      
            except:
                print("Response not in valid JSON format")
    except:
        print("Something went wrong with requests.get")
  



In [123]:
def get_lat_lng(destination):
    url="https://maps.googleapis.com/maps/api/geocode/json?address=%s&key=%s" % (destination,api_key)
    try:
        response = requests.get(url)
        if not response.status_code == 200:
            print("HTTP error",response.status_code)
        else:
            try:
                response_data = response.json()
                
                return (response_data['results'][0]["geometry"]["location"]["lng"],
                       response_data['results'][0]["geometry"]["location"]["lat"])

            except:
                print("Response not in valid JSON format")
    except:
        print("Something went wrong with requests.get")

In [124]:
def get_info_row(row):
    # call your function to get the distance, duration, longitude, and latitude for the target in the row
    distance, duration = Distance_Duration(row['Target'])
    longitude, latitude = get_lat_lng(row['Target'])
    
    # return a Series with the distance, duration, longitude, and latitude
    return pd.Series({'Distance_km': distance, 'Duration (hour + minutes)': duration, 'Longitude': longitude, 'Latitude': latitude})

# apply the get_distance_info_row function to each row of the Target column, and concatenate the results with the original dataframe
df = pd.concat([df, df.apply(get_info_row, axis=1)], axis=1)


In [125]:
df

,Target,Distance_km,Duration (hour + minutes),Longitude,Latitude
0,Istanbul,1815.227,21 hours 2 minutes,28.978359,41.008238
1,Amsterdam,4532.541,48 hours 3 minutes,4.904139,52.367573
2,Valletta,3792.883,50 hours 52 minutes,14.514100,35.899237
3,Basel,4092.805,44 hours 2 minutes,7.588576,47.559599
4,Doha,2164.477,22 hours 38 minutes,51.531040,25.285447


## 3 Cities Furthest From Tel Aviv - From Top to Bottom 

In [126]:
top_three = df.nlargest(3, 'Distance_km').reset_index(drop=True)
top_three.index = top_three.index + 1
top_three

,Target,Distance_km,Duration (hour + minutes),Longitude,Latitude
1,Amsterdam,4532.541,48 hours 3 minutes,4.904139,52.367573
2,Basel,4092.805,44 hours 2 minutes,7.588576,47.559599
3,Valletta,3792.883,50 hours 52 minutes,14.514100,35.899237
